In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv2d_to_matmul(input, conv_layer):
    # Extract Conv2D parameters
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    # Unfold (im2col) the input tensor
    input_unf = F.unfold(input, kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)

    # Reshape the weight tensor of the conv layer
    weight = conv_layer.weight.view(out_channels, -1)

    # Perform matrix multiplication
    output_unf = weight @ input_unf

    # Reshape the output to the correct dimensions
    output_height = (input.size(2) + 2*padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input.size(3) + 2*padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    output = output_unf.view(1, out_channels, output_height, output_width)

    return output

# Example usage
input = torch.randn(1, 3, 5, 5)  # Batch size 1, 3 channels, 5x5 image
conv_layer = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)

# Get the output using the conv2d_to_matmul function
output = conv2d_to_matmul(input, conv_layer)

# Verify against the standard convolution
conv_output = conv_layer(input)

print("Output from matrix multiplication:")
print(output)

print("Output from standard convolution:")
print(conv_output)

# Verify that the outputs are the same
print("Difference between outputs:", torch.abs(output - conv_output).max())


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv2d_to_matmul(input, conv_layer):
    # Extract Conv2D parameters
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    # Unfold (im2col) the input tensor
    input_unf = F.unfold(input, kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)

    # Reshape the weight tensor of the conv layer
    weight = conv_layer.weight.view(out_channels, -1)

    # Perform matrix multiplication
    output_unf = weight @ input_unf

    # Add the bias
    if conv_layer.bias is not None:
        output_unf += conv_layer.bias.unsqueeze(1)

    # Reshape the output to the correct dimensions
    output_height = (input.size(2) + 2*padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input.size(3) + 2*padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    output = output_unf.view(1, out_channels, output_height, output_width)

    return output

# Example usage
input = torch.randn(1, 3, 5, 5)  # Batch size 1, 3 channels, 5x5 image
conv_layer = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)

# Get the output using the conv2d_to_matmul function
output = conv2d_to_matmul(input, conv_layer)

# Verify against the standard convolution
conv_output = conv_layer(input)

print("Output from matrix multiplication:")
print(output)

print("Output from standard convolution:")
print(conv_output)

# Verify that the outputs are the same
print("Difference between outputs:", torch.abs(output - conv_output).max())


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv2d_to_matmul(input, conv_layer):


    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

 
    input_unf = F.unfold(input, kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)

    weight = conv_layer.weight.view(out_channels, -1)


    output_unf = weight @ input_unf

    if conv_layer.bias is not None:
        output_unf += conv_layer.bias.unsqueeze(1)

    output_height = (input.size(2) + 2*padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input.size(3) + 2*padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    output = output_unf.view(1, out_channels, output_height, output_width)

    return output


with torch.no_grad():
    input = torch.randn(1, 3, 55, 55)  
    conv_layer = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3, stride=1, padding=1)

    output = conv2d_to_matmul(input, conv_layer)


    conv_output = conv_layer(input)

print("Output from matrix multiplication:")
print(output)

print("Output from standard convolution:")
print(conv_output)


print("Difference between outputs:", torch.abs(output - conv_output).max())

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd.functional import jacobian

def spectral_norm(matrix):
    u, s, v = torch.svd(matrix, some=True)
    return s.max()

def lipschitz_constant(network, input_shape):
    lipschitz_constants = []

    def register_hook(layer):
        if isinstance(layer, (nn.Linear, nn.Conv2d)):
            def hook(module, input, output):
                if isinstance(module, nn.Linear):
                    weight = module.weight.data
                elif isinstance(module, nn.Conv2d):
                    weight = module.weight.data.view(module.out_channels, -1)
                lipschitz_constants.append(spectral_norm(weight))
            return layer.register_forward_hook(hook)

    hooks = []
    for layer in network.modules():
        hook = register_hook(layer)
        if hook:
            hooks.append(hook)

    input_tensor = torch.randn(*input_shape)
    network(input_tensor)


    for hook in hooks:
        hook.remove()


    total_lipschitz_constant = torch.prod(torch.tensor(lipschitz_constants))

    return total_lipschitz_constant.item()


In [ ]:
import torchvision.models as models

model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
input_shape = (1, 3, 224, 224
               ) 
lipschitz_const = lipschitz_constant(model, input_shape)
print(f"Constante de Lipschitz du réseau: {lipschitz_const}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

def find_maximizing_input(network, target_class, input_shape, num_iterations=300, learning_rate=0.01):
 
    input_tensor = torch.randn(input_shape, requires_grad=True)

    optimizer = optim.Adam([input_tensor], lr=learning_rate)

    for iteration in range(num_iterations):
        optimizer.zero_grad()

        output = network(input_tensor)

        target_output = output[0, target_class]

        loss = -target_output

     
        loss.backward()

        optimizer.step()

        with torch.no_grad():
            input_tensor.clamp_(0, 1)  

        if iteration % 10 == 0:
            print(f"Iteration {iteration}, Loss: {loss.item()}")

    return input_tensor

vgg19 = models.vgg19(pretrained=True)

input_shape = (1, 3, 224, 224)
target_class = 1

maximizing_input = find_maximizing_input(vgg19, target_class, input_shape)

print("Tenseur d'entrée qui maximise la sortie:")
print(maximizing_input)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
maximizing_input_np = maximizing_input_1_t.squeeze(0).detach().numpy()
maximizing_input_np = np.transpose(maximizing_input_np, (1, 2, 0))

# Afficher l'image qui maximise la sortie du réseau
plt.imshow(maximizing_input_np)
plt.title("Image qui maximise la projection sur les valeurs singulières du réseau")
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

def find_maximizing_input(network, num_iterations=100, learning_rate=0.01,input_tensor = torch.randn(3,224,224),ortho = 1,svd_rank = 0):
    # Initialiser une entrée aléatoire

    #input_tensor = maximizing_input_2

    # Définir l'optimiseur
    optimizer = optim.Adam([input_tensor], lr=learning_rate)

    for iteration in range(num_iterations):
        optimizer.zero_grad()

        # Calculer la sortie du réseau
        output = input_tensor
        projections = []

        for layer in network.children():
            if isinstance(layer, nn.Conv2d):
                weight = layer.weight.data.view(layer.out_channels, -1)
                u, s, v = torch.svd(weight)
                v1 = v[:, svd_rank] 
                output_unfolded = F.unfold(output, kernel_size=layer.kernel_size, padding=layer.padding, stride=layer.stride)
                projection = torch.matmul(v1, output_unfolded)
                projections.append(projection)
                output = layer(output)
            elif isinstance(layer, nn.Linear):
                weight = layer.weight.data
                u, s, v = torch.svd(weight)
                v1 = v[:, svd_rank]  
                projection = torch.matmul(v1, output.view(output.size(0), -1).t())
                projections.append(projection)
                output = layer(output.view(output.size(0), -1))
            elif isinstance(layer, nn.Sequential) or isinstance(layer, nn.ReLU) or isinstance(layer, nn.MaxPool2d):
                output = layer(output)
            else:
                raise NotImplementedError(f"Layer type {type(layer)} is not supported.")

        total_projection = sum(torch.norm(p) for p in projections)

      
        loss = -ortho*total_projection

        loss.backward()

       
        optimizer.step()

        with torch.no_grad():
            input_tensor.clamp_(0, 1) 

        if iteration % 10 == 0:
            print(f"Iteration {iteration}, Loss: {loss.item()}")

    return input_tensor


vgg19 = models.vgg19(pretrained=True)


input_shape = (1, 3, 224, 224)

input_tensor_0 = torch.randn(3,224,224, requires_grad=True)
#maximizing_input_0 = find_maximizing_input(vgg19.features, input_tensor = input_tensor, ortho = 1,svd_rank=0)
#maximizing_input_0_t = find_maximizing_input(vgg19.features, input_tensor = input_tensor, ortho = -1,svd_rank=0)
maximizing_input_1 = find_maximizing_input(vgg19.features, input_tensor = input_tensor_0, ortho = 1,svd_rank=1)
maximizing_input_1_t = find_maximizing_input(vgg19.features, input_tensor = input_tensor_0, ortho = -1,svd_rank=1)
maximizing_input_2 = find_maximizing_input(vgg19.features, input_tensor = input_tensor_0, ortho = 1,svd_rank=2)
maximizing_input_2_t = find_maximizing_input(vgg19.features, input_tensor = input_tensor_0, ortho = -1,svd_rank=2)


In [ ]:
torch.max(model(maximizing_input))


In [ ]:
torch.max(maximizing_input)

In [ ]:
import torch
u = torch.randn(10000,3,52,52)
x = torch.where(u>0.5,u,0).to_sparse()
y = torch.where(u>0.5,u,0).to_sparse()

(u[0]*x).to_dense().shape



In [ ]:
z=x+y

In [ ]:
def dim_chunk(x, available_RAM):
        dense_memory_footprint = torch.prod(torch.tensor(x.shape)) *4/1e9
        return max(1,available_RAM//(4*dense_memory_footprint))
x = torch.randn(1000,1000,100
            )
dim_chunk(x,100)

In [ ]:
def conv2d_to_matmul(input, conv_layer):
 
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation


    input_unf = F.unfold(input, kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)


    weight = conv_layer.weight.view(out_channels, -1)

  
    output_unf = weight @ input_unf


    output_height = (input.size(2) + 2*padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input.size(3) + 2*padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    output = output_unf.view(1, out_channels, output_height, output_width)

    return output

input = torch.randn(1, 3, 32, 32)  
conv_layer = torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  #

output = conv2d_to_matmul(input, conv_layer)
print(output.shape)  #
print(torch.max(output -conv_layer(input)))

In [ ]:
import torch
import torch.nn.functional as F

def conv2d_to_matmul(input, conv_layer):
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    output_height = (input.size(2) + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input.size(3) + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1

    weight = conv_layer.weight.view(out_channels, in_channels, -1)
    weight = weight.permute(1, 2, 0).contiguous().view(-1, out_channels)

    input_unf = F.unfold(input, kernel_size=kernel_size, dilation=dilation, padding=padding, stride=stride)
    

    output_unf = weight.t() @ input_unf

    output = output_unf.view(1, out_channels, output_height, output_width)

    return output

input = torch.randn(1, 3, 32, 32)  
conv_layer = torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  #

output = conv2d_to_matmul(input, conv_layer)
print(output.shape)  #
print(torch.max(output -conv_layer(input)))

In [ ]:
import torch
import torch.nn.functional as F

def sparse_conv2d_to_matmul(input_sparse, conv_layer):
    assert input_sparse.layout == torch.sparse_coo, "L'entrée doit être un tenseur sparse."

    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    indices = input_sparse._indices()
    values = input_sparse._values()

    # Calculer la taille de la sortie
    input_height, input_width = input_sparse.size(2), input_sparse.size(3)
    output_height = (input_height + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input_width + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    
    # Initialiser la sortie sparse
    output = torch.zeros((1, out_channels, output_height, output_width), device=input_sparse.device)

    # Extraire les poids du noyau
    weight = conv_layer.weight.view(out_channels, in_channels, kernel_size[0], kernel_size[1])

    # Appliquer la convolution manuellement
    for i in range(indices.size(1)):
        b, c, h, w = indices[:, i]
        if h >= padding[0] and h < input_height + padding[0] and w >= padding[1] and w < input_width + padding[1]:
            for kh in range(kernel_size[0]):
                for kw in range(kernel_size[1]):
                    if (h - kh * dilation[0] + padding[0]) % stride[0] == 0 and (w - kw * dilation[1] + padding[1]) % stride[1] == 0:
                        h_out = (h - kh * dilation[0] + padding[0]) // stride[0]
                        w_out = (w - kw * dilation[1] + padding[1]) // stride[1]
                        if h_out >= 0 and h_out < output_height and w_out >= 0 and w_out < output_width:
                            output[0, :, h_out, w_out] += weight[:, c, kh, kw] * values[i]

    return output

# Exemple d'utilisation
input_sparse = torch.sparse_coo_tensor(
    indices=[[0, 0, 0, 1], [0, 1, 2, 2], [2, 3, 2, 2], [2, 2, 4, 2]],
    values=[1.0, 2.0, 3.0, 4.0],
    size=(10, 3, 32, 32)
)  # Exemple de tenseur d'entrée sparse

conv_layer = torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Exemple de couche de convolution
conv_layer.bias.data = torch.zeros_like(conv_layer.bias.data)

output = sparse_conv2d_to_matmul(input_sparse, conv_layer)
print(output.shape)  # Devrait correspondre à la sortie d'une couche de convolution
output_2 = conv_layer(input_sparse.to_dense())
print(torch.max(output-output_2))

In [ ]:
output_2-output

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Conv2d

def construct_convolution_matrix(conv_layer, input_size):
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    input_height, input_width = input_size
    output_height = (input_height + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input_width + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1
    
    weight = conv_layer.weight.view(out_channels, in_channels * kernel_size[0] * kernel_size[1])

    # Matrice de convolution C
    C = torch.zeros(out_channels * output_height * output_width, in_channels * input_height * input_width)

    for oh in range(output_height):
        for ow in range(output_width):
            for kh in range(kernel_size[0]):
                for kw in range(kernel_size[1]):
                    for ic in range(in_channels):
                        ih = oh * stride[0] - padding[0] + kh * dilation[0]
                        iw = ow * stride[1] - padding[1] + kw * dilation[1]
                        if 0 <= ih < input_height and 0 <= iw < input_width:
                            for oc in range(out_channels):
                                C[oc * output_height * output_width + oh * output_width + ow, 
                                  ic * input_height * input_width + ih * input_width + iw] = weight[oc, ic * kernel_size[0] * kernel_size[1] + kh * kernel_size[1] + kw]

    return C, output_height, output_width

def conv2d_to_matmul(input, conv_layer):
    input_size = input.size()[2:]

    C, output_height, output_width = construct_convolution_matrix(conv_layer, input_size)
    
    
    
    output_flat = C * input
    
    
    return output

# Exemple d'utilisation
input = torch.randn(10, 3, 32, 32)
input = torch.where(input>0.5,input, 0).to_sparse()  # Exemple d'entrée
conv_layer = Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Exemple de couche de convolution

output = conv2d_to_matmul(input, conv_layer)
print(output.shape)  # Devrait correspondre à la sortie d'une couche de convolution


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Conv2d

def construct_convolution_matrix(conv_layer, input_size):
    in_channels = conv_layer.in_channels
    out_channels = conv_layer.out_channels
    kernel_size = conv_layer.kernel_size
    stride = conv_layer.stride
    padding = conv_layer.padding
    dilation = conv_layer.dilation

    input_height, input_width = input_size
    output_height = (input_height + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) // stride[0] + 1
    output_width = (input_width + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) // stride[1] + 1

    # Initialiser la matrice de convolution C
    C = torch.zeros(out_channels * output_height * output_width, in_channels * input_height * input_width)

    # Extraire les poids du noyau
    weight = conv_layer.weight.view(out_channels, in_channels * kernel_size[0] * kernel_size[1])

    # Remplir la matrice de convolution C
    for oc in range(out_channels):
        for ic in range(in_channels):
            for kh in range(kernel_size[0]):
                for kw in range(kernel_size[1]):
                    for oh in range(output_height):
                        for ow in range(output_width):
                            ih = oh * stride[0] - padding[0] + kh * dilation[0]
                            iw = ow * stride[1] - padding[1] + kw * dilation[1]
                            if 0 <= ih < input_height and 0 <= iw < input_width:
                                C[oc * output_height * output_width + oh * output_width + ow, 
                                  ic * input_height * input_width + ih * input_width + iw] = weight[oc, ic * kernel_size[0] * kernel_size[1] + kh * kernel_size[1] + kw]
    return C, output_height, output_width

def conv2d_to_matmul(input, conv_layer):
    input_size = input.size()[2:]
    C, output_height, output_width = construct_convolution_matrix(conv_layer, input_size)
    
    input_flat = input.view(-1)
    
    output_flat = C @ input_flat
    
    output = output_flat.view(1, conv_layer.out_channels, output_height, output_width)
    
    return output

# Exemple d'utilisation
input = torch.randn(10, 3, 32, 32)  # Exemple d'entrée
conv_layer = Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # Exemple de couche de convolution

output = conv2d_to_matmul(input, conv_layer)
print(output.shape)  # Devrait correspondre à la sortie d'une couche de convolution


In [ ]:
import torch
import torch.nn as nn
import copy
import torch.nn.functional as F

class UnStackNetwork:
    def __init__(self, model, input_dim):
        self.model = model
        self.input_dim = input_dim
        self.output = {}
        self.unstack_network()

    def unstack_network(self):
        x = torch.randn(1, *self.input_dim)
        for name, module in self.model.named_children():
            if isinstance(module, nn.Sequential):
                for layer_name, layer in module.named_children():
                    x = self.process_layer(f"{name}_{layer_name}", layer, x)
            else:
                x = self.process_layer(name, module, x)

    def process_layer(self, name, layer, x):
        if isinstance(layer, nn.Linear):
            # Flatten the tensor before passing to fully connected layers
            self.process_flatten(name, nn.Flatten(), x)
            x = nn.Flatten()(x)
            self.process_linear_layer(name, layer, x)
        elif isinstance(layer, nn.Conv2d):
            self.process_conv_layer(name, layer, x)
        elif isinstance(layer, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d)):
            self.process_activation_layer(name, layer)
        x = layer(x) if layer is not None else x  # Handle input layer without layer
        return x

    def process_linear_layer(self, name, layer, x):
        self.output[name] = {
            'type': type(layer),
            'original': copy.deepcopy(layer),
            'epsilon_{}'.format(name): self.copy_with_zero_bias(layer),
            'noise_{}'.format(name): self.copy_with_abs_weights(layer),
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_flatten(self, name, layer, x):
        self.output[f'{name}_flatten'] = {
            'type': type(layer),
            'original': layer,
            'epsilon_{}'.format(f'{name}_flatten'): layer,
            'noise_{}'.format(f'{name}_flatten'): layer,
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_conv_layer(self, name, layer, x):
        self.output[name] = {
            'type': type(layer),
            'original': copy.deepcopy(layer),
            'epsilon_{}'.format(name): self.copy_with_zero_bias(layer),
            'noise_{}'.format(name): self.copy_with_abs_weights(layer),
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_activation_layer(self, name, layer):
        self.output[name] = {
            'activation': layer.__class__.__name__
        }

    def copy_with_zero_bias(self, layer):
        new_layer = copy.deepcopy(layer)
        if isinstance(layer, (nn.Linear, nn.Conv2d)):
            with torch.no_grad():
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
        return new_layer

    def copy_with_abs_weights(self, layer):
        new_layer = copy.deepcopy(layer)
        with torch.no_grad():
            if isinstance(layer, (nn.Linear, nn.Conv2d)):
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
                new_layer.weight.abs_()
        return new_layer

    def compute_output_dim(self, layer, x):
        with torch.no_grad():
            out = layer(x) if layer is not None else x  
        return out.shape

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
 
        self.fc1 = nn.Linear(in_features=32768, out_features=128)  
        self.fc2 = nn.Linear(in_features=128, out_features=10)  
        self.relu4 = nn.ReLU()

    def forward(self, x):

        x = self.relu1(self.conv1(x))

        x = self.relu2(self.conv2(x))
 

  
        x = x.view(x.size(0), -1)
    
        x = self.relu3(self.fc1(x))
      
        x = self.relu4(self.fc2(x))
        return x

class SimpleCNN_2(nn.Module):
    def __init__(self):
        super(SimpleCNN_2, self).__init__()
        # Deux couches de convolution suivies de ReLU
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        
        # Deux couches entièrement connectées suivies de ReLU
        self.fc1 = nn.Linear(in_features=32768, out_features=128)  # Supposons que la taille de l'entrée est 32x32
        self.fc2 = nn.Linear(in_features=128, out_features=10)  # Supposons 10 classes de sortie

    def forward(self, x):
        # Appliquer la première couche de convolution et ReLU
        x = F.relu(self.conv1(x))
        # Appliquer la deuxième couche de convolution et ReLU
        x = F.relu(self.conv2(x))
        # Appliquer le pooling pour réduire la taille (2x2 pooling)
     
        # Aplatir le tenseur pour l'entrée dans la couche entièrement connectée
        x = x.view(x.size(0), -1)
        # Appliquer la première couche entièrement connectée et ReLU
        x = F.relu(self.fc1(x))
        # Appliquer la deuxième couche entièrement connectée
        x = self.fc2(x)
        return x



# Instancier et utiliser la classe
input_tensor = torch.randn(1, 3, 32, 32)  # Exemple de tenseur d'entrée
model = SimpleCNN_2()
r1 = model(input_tensor)
torch.save(model, 'model_complete.pth')
del model
model= torch.load('model_complete.pth')
print(model)
r2 = model(input_tensor)
processor = UnStackNetwork(model, input_tensor.shape[1:])
print(processor.output)  # Afficher les résultats pour vérifier
print(r1-r2)

In [ ]:
for name, module in model.named_modules():
    print(f"Layer: {name}, Type: {module.__class__.__name__}")

In [ ]:
print(model.forward)

In [ ]:
class ActivationLogger(nn.Module):
    def __init__(self, model):
        super(ActivationLogger, self).__init__()
        self.model = model
        self.activations = []
        self.layers = []

    def forward(self, x):
        for name, module in self.model.named_children():
            x = module(x)
            self.layers.append((name, module))
            if isinstance(module, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d)):
                self.activations.append((name, module))
            elif isinstance(module, nn.Sequential):
                for layer_name, layer in module.named_children():
                    x = layer(x)
                    self.layers.append((f"{name}_{layer_name}", layer))
                    if isinstance(layer, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d)):
                        self.activations.append((f"{name}_{layer_name}", layer))
        return x


In [ ]:

model=torch.load('model_complete.pth')

# Wrap the model
logger = ActivationLogger(model)

# Forward pass to log activations
input_tensor = torch.randn(1, 3, 32, 32)
output = logger(input_tensor)

# Retrieve and print the structure and activations
print("Layers:")
for name, layer in logger.layers:
    print(f"{name}: {layer}")

print("\nActivations:")
for name, activation in logger.activations:
    print(f"{name}: {activation}")


In [ ]:
import torch
import torch.nn as nn

# Supposons que vous ayez un fichier 'network.pth' contenant le modèle
model = torch.load('model_complete.pth')

# Vérifiez que le modèle est une instance de nn.Module
assert isinstance(model, nn.Module), "Le modèle chargé n'est pas une instance de nn.Module"

# Définissez une classe wrapper
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        # Ajoutez ici tout traitement que vous souhaitez avant d'appeler la méthode forward d'origine
        print("Avant l'appel de forward")
        
        # Appelez la méthode forward d'origine
        output = self.model.forward(*args, **kwargs)
        
        # Ajoutez ici tout traitement que vous souhaitez après l'appel de la méthode forward d'origine
        print("Après l'appel de forward")
        
        return output

# Créez une instance du wrapper avec le modèle chargé
wrapped_model = ModelWrapper(model)

# Utilisez le modèle wrappé comme d'habitude
input_tensor = torch.randn(1, 3, 32, 32)  # Exemple de tenseur d'entrée
output = wrapped_model(input_tensor)

print(output)


In [ ]:
print(wrapped_model)

In [ ]:
import inspect
# Analyse de la méthode forward
forward_method = model.forward
source_code = inspect.getsource(forward_method)

print("Code source de la méthode forward :")
print(source_code)

# Afficher des informations supplémentaires
signature = inspect.signature(forward_method)
print("\nSignature de la méthode forward :")
print(signature)

docstring = inspect.getdoc(forward_method)
print("\nDocstring de la méthode forward :")
print(docstring)

In [ ]:
source_code

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import copy


class UnStackNetwork:
    def __init__(self, model, input_dim):
        self.model = model
        self.input_dim = input_dim
        self.output = {}
        self.unstack_network()

    def unstack_network(self):
        x = torch.randn(1, *self.input_dim)
        for name, module in self.model.named_children():
            if isinstance(module, nn.Sequential):
                for layer_name, layer in module.named_children():
                    x = self.process_layer(f"{name}_{layer_name}", layer, x)
            else:
                x = self.process_layer(name, module, x)

    def process_layer(self, name, layer, x):
        if isinstance(layer, nn.Linear):
            # Flatten the tensor before passing to fully connected layers
            self.process_flatten(name, nn.Flatten(), x)
            x = nn.Flatten()(x)
            self.process_linear_layer(name, layer, x)
        elif isinstance(layer, nn.Conv2d):
            self.process_conv_layer(name, layer, x)
        elif isinstance(layer, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d, nn.BatchNorm2d, nn.Dropout)):
            self.process_activation_layer(name, layer, x)
        x = layer(x) if layer is not None else x  # Handle input layer without layer
        return x

    def process_linear_layer(self, name, layer, x):
        self.output[name] = {
            'type': type(layer),
            'original': copy.deepcopy(layer),
            'epsilon_{}'.format(name): self.copy_with_zero_bias(layer),
            'noise_{}'.format(name): self.copy_with_abs_weights(layer),
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_flatten(self, name, layer, x):
        self.output[f'{name}_flatten'] = {
            'type': type(layer),
            'original': layer,
            'epsilon_{}'.format(f'{name}_flatten'): layer,
            'noise_{}'.format(f'{name}_flatten'): layer,
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_conv_layer(self, name, layer, x):
        self.output[name] = {
            'type': type(layer),
            'original': copy.deepcopy(layer),
            'epsilon_{}'.format(name): self.copy_with_zero_bias(layer),
            'noise_{}'.format(name): self.copy_with_abs_weights(layer),
            'output_dim': self.compute_output_dim(layer, x)
        }

    def process_activation_layer(self, name, layer, x):
        self.output[name] = {
            'activation': layer.__class__.__name__,
            'output_dim': self.compute_output_dim(layer, x)
        }

    def copy_with_zero_bias(self, layer):
        new_layer = copy.deepcopy(layer)
        if isinstance(layer, (nn.Linear, nn.Conv2d)):
            with torch.no_grad():
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
        return new_layer

    def copy_with_abs_weights(self, layer):
        new_layer = copy.deepcopy(layer)
        with torch.no_grad():
            if isinstance(layer, (nn.Linear, nn.Conv2d)):
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
                new_layer.weight.abs_()
        return new_layer

    def compute_output_dim(self, layer, x):
        with torch.no_grad():
            out = layer(x) if layer is not None else x
        return out.shape


# Exemple d'utilisation
model = torch.load('model_complete.pth')
input_dim = (3, 32, 32)  # Dimensions de l'entrée pour ResNet18
unstacked_network = UnStackNetwork(model, input_dim)

# Afficher la sortie
for name, details in unstacked_network.output.items():
    print(f"{name}: {details}")


In [ ]:
forward_method = model.forward
source_code = inspect.getsource(forward_method)
print(source_code)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy

class UnStackNetwork:
    def __init__(self, model, input_dim):
        self.model = model
        self.input_dim = input_dim
        self.output = {}
        self.handles = []
        self.register_hooks()
        self.unstack_network()
        self.remove_hooks()

    def register_hooks(self):
        def hook(module, input, output, name):
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                self.output[name] = {
                    'type': type(module),
                    'original': copy.deepcopy(module),
                    'epsilon_{}'.format(name): self.copy_with_zero_bias(module),
                    'noise_{}'.format(name): self.copy_with_abs_weights(module),
                    'output_dim': output.shape
                }
            elif isinstance(module, nn.Flatten):
                self.output[name] = {
                    'type': type(module),
                    'original': module,
                    'epsilon_{}'.format(name): module,
                    'noise_{}'.format(name): module,
                    'output_dim': output.shape
                }
            elif isinstance(module, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d, nn.BatchNorm2d, nn.Dropout)):
                self.output[name] = {
                    'activation': module.__class__.__name__,
                    'output_dim': output.shape
                }

        for name, module in self.model.named_modules():
            handle = module.register_forward_hook(lambda module, input, output, name=name: hook(module, input, output, name))
            self.handles.append(handle)

    def unstack_network(self):
        x = torch.randn(1, *self.input_dim)
        self.model(x)

    def remove_hooks(self):
        for handle in self.handles:
            handle.remove()

    def copy_with_zero_bias(self, layer):
        new_layer = copy.deepcopy(layer)
        if isinstance(layer, (nn.Linear, nn.Conv2d)):
            with torch.no_grad():
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
        return new_layer

    def copy_with_abs_weights(self, layer):
        new_layer = copy.deepcopy(layer)
        with torch.no_grad():
            if isinstance(layer, (nn.Linear, nn.Conv2d)):
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
                new_layer.weight.abs_()
        return new_layer

# Exemple d'utilisation
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomModel()
input_dim = (3, 32, 32)
unstacked_network = UnStackNetwork(model, input_dim)

# Afficher la sortie
for name, details in unstacked_network.output.items():
    print(f"{name}: {details}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy

class UnStackNetwork:
    def __init__(self, model, input_dim):
        self.model = model
        self.input_dim = input_dim
        self.output = {}
        self.handles = []
        self.layers = []
        self.register_hooks()
        self.unstack_network()
        self.remove_hooks()

    def register_hooks(self):
        def hook(module, input, output, name):
            self.layers.append((name, module, output))
            if isinstance(module, (nn.Conv2d, nn.Linear, nn.Flatten)):
                self.output[name] = {
                    'type': type(module),
                    'original': copy.deepcopy(module),
                    'epsilon_{}'.format(name): self.copy_with_zero_bias(module),
                    'noise_{}'.format(name): self.copy_with_abs_weights(module),
                    'output_dim': output.shape
                }
            elif isinstance(module, (nn.ReLU, nn.Sigmoid, nn.Tanh, nn.AdaptiveAvgPool2d, nn.MaxPool2d, nn.BatchNorm2d, nn.Dropout)):
                self.output[name] = {
                    'activation': module.__class__.__name__,
                    'output_dim': output.shape
                }

        for name, module in self.model.named_modules():
            if not isinstance(module, nn.Sequential):  # Ignorer les séquentiels pour éviter les doublons
                handle = module.register_forward_hook(lambda module, input, output, name=name: hook(module, input, output, name))
                self.handles.append(handle)

    def unstack_network(self):
        x = torch.randn(1, *self.input_dim)
        self.model(x)

    def remove_hooks(self):
        for handle in self.handles:
            handle.remove()

    def copy_with_zero_bias(self, layer):
        new_layer = copy.deepcopy(layer)
        if isinstance(layer, (nn.Linear, nn.Conv2d)):
            with torch.no_grad():
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
        return new_layer

    def copy_with_abs_weights(self, layer):
        new_layer = copy.deepcopy(layer)
        with torch.no_grad():
            if isinstance(layer, (nn.Linear, nn.Conv2d)):
                if new_layer.bias is not None:
                    new_layer.bias.zero_()
                new_layer.weight.abs_()
        return new_layer

# Exemple d'utilisation
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 10)
        self.flatten = nn.Flatten()


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CustomModel()
input_dim = (3, 32, 32)
unstacked_network = UnStackNetwork(model, input_dim)

# Afficher la sortie
for name, details in unstacked_network.output.items():
    print(f"{name}: {details}")


In [ ]:
!pip install onnx2torch


In [ ]:
import torch
import onnx
from onnx2torch import convert
path = './vgg16-12.onnx'
onnx_model = onnx.load(path)
target_version = 7
pytorch_model = convert(onnx_model,target_version)
pytorch_model = pytorch_model.eval()

pytorch_model(torch.randn(1,3,224,224))

In [ ]:
import inspect
forward_method = pytorch_model.forward
source_code = inspect.getsource(forward_method)
print(source_code)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def im2col(input, kernel_size, stride=1, padding=0):
    # Apply padding
    input_padded = F.pad(input, (padding, padding, padding, padding))
    
    # Extract the dimensions
    batch_size, channels, height, width = input_padded.size()
    out_height = (height - kernel_size) // stride + 1
    out_width = (width - kernel_size) // stride + 1

    col = torch.zeros(batch_size, channels, kernel_size, kernel_size, out_height, out_width)
    for y in range(kernel_size):
        y_max = y + stride * out_height
        for x in range(kernel_size):
            x_max = x + stride * out_width
            col[:, :, y, x, :, :] = input_padded[:, :, y:y_max:stride, x:x_max:stride]

    col = col.permute(0, 4, 5, 1, 2, 3).contiguous()
    col = col.view(batch_size, out_height * out_width, -1)
    return col

class Conv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def conv_to_matmul(self, x):
        batch_size, in_channels, height, width = x.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # im2col transformation
        x_col = im2col(x, self.kernel_size, self.stride, self.padding)
        x_col = x_col.view(batch_size, -1, self.kernel_size * self.kernel_size * self.in_channels)

        # Reshape weight
        weight_col = self.weight.view(self.out_channels, -1)

        # Matrix multiplication
        out = weight_col @ x_col.transpose(1, 2)
        if self.bias is not None:
            out += self.bias.view(1, -1, 1)

        # Reshape back to the output shape
        out = out.view(batch_size, self.out_channels, out_height, out_width)
        return out

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 2
    kernel_size = 3
    stride = 1
    padding = 1

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul
    conv_to_matmul = Conv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée
    input_tensor = torch.randn(1, in_channels, 5, 5)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor)
    print("Résultat de la convolution:", conv_output)

    # Passage de l'entrée à travers la transformation matmul
    matmul_output = conv_to_matmul.conv_to_matmul(input_tensor)
    print("Résultat du produit matriciel:", matmul_output)

    # Vérification de l'égalité des résultats
    assert torch.allclose(conv_output, matmul_output, atol=1e-6), "Les résultats ne correspondent pas!"
    print("Les résultats correspondent.")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            for oc in range(self.out_channels):
                conv_value = 0.0
                for ic in range(self.in_channels):
                    for ky in range(self.kernel_size):
                        for kx in range(self.kernel_size):
                            yy = y + ky - self.kernel_size // 2
                            xx = x + kx - self.kernel_size // 2
                            if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                weight = self.weight[oc, ic, ky, kx].item()
                                if weight != 0:
                                    input_value = x_sparse[0, ic, yy, xx].item()
                                    conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                output_indices.append([b.item(), oc, (y - self.kernel_size // 2) // self.stride, (x - self.kernel_size // 2) // self.stride])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 2
    kernel_size = 3
    stride = 1
    padding = 1

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            for oc in range(self.out_channels):
                conv_value = 0.0
                for ic in range(self.in_channels):
                    for ky in range(self.kernel_size):
                        for kx in range(self.kernel_size):
                            yy = y + ky - self.kernel_size // 2
                            xx = x + kx - self.kernel_size // 2
                            if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                weight = self.weight[oc, ic, ky, kx].item()
                                input_value = x_sparse[0, ic, yy, xx].item() if (0 <= yy < height and 0 <= xx < width) else 0
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                output_indices.append([b.item(), oc, (y - self.kernel_size // 2) // self.stride, (x - self.kernel_size // 2) // self.stride])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 2

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5,5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)
    conv_output =conv_layer(input_tensor_dense)
    diff = torch.sum(matmul_output_sparse.to_dense()-conv_output)
    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
diff

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            for oc in range(self.out_channels):
                conv_value = 0.0
                for ic in range(self.in_channels):
                    for ky in range(self.kernel_size):
                        for kx in range(self.kernel_size):
                            yy = y + ky - self.kernel_size // 2
                            xx = x + kx - self.kernel_size // 2
                            if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                weight = self.weight[oc, ic, ky, kx].item()
                                input_value = values[i] if (yy == y and xx == x and ic == c) else 0
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 2

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            for oc in range(self.out_channels):
                conv_value = 0.0
                for ic in range(self.in_channels):
                    for ky in range(self.kernel_size):
                        for kx in range(self.kernel_size):
                            yy = y + ky - self.kernel_size // 2
                            xx = x + kx - self.kernel_size // 2
                            if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                weight = self.weight[oc, ic, ky, kx].item()
                                input_value = values[(b == indices[0]) & (ic == indices[1]) & (yy == indices[2]) & (xx == indices[3])].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 2

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_im2col(self, indices, values, height, width, kernel_size, stride=1, padding=0):
        # Apply padding to the indices
        padded_indices = indices.clone()
        padded_indices[2:] += padding

        # Calculate output dimensions
        out_height = (height + 2 * padding - kernel_size) // stride + 1
        out_width = (width + 2 * padding - kernel_size) // stride + 1

        col_indices = []
        col_values = []

        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < kernel_size // 2 or y >= height + padding + kernel_size // 2:
                continue
            if x < kernel_size // 2 or x >= width + padding + kernel_size // 2:
                continue

            for ky in range(kernel_size):
                for kx in range(kernel_size):
                    yy = y + ky - kernel_size // 2
                    xx = x + kx - kernel_size // 2
                    if 0 <= yy < height + 2 * padding and 0 <= xx < width + 2 * padding:
                        col_indices.append([b.item(), c.item(), ky, kx, yy // stride, xx // stride])
                        col_values.append(values[i].item())

        col_indices = torch.tensor(col_indices).t()
        col_values = torch.tensor(col_values)

        return col_indices, col_values

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()

        # Perform sparse im2col
        col_indices, col_values = self.sparse_im2col(indices, values, height, width, self.kernel_size, self.stride, self.padding)

        # Convert im2col result to sparse tensor
        col_shape = (batch_size, in_channels, self.kernel_size, self.kernel_size, out_height, out_width)
        col_sparse = torch.sparse.FloatTensor(col_indices, col_values, col_shape)

        # Reshape weight
        weight_col = self.weight.view(self.out_channels, -1)

        # Perform matrix multiplication
        out_sparse = torch.sparse.mm(weight_col, col_sparse.view(in_channels * self.kernel_size * self.kernel_size, -1))

        # Reshape output to the correct shape
        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_indices = out_sparse._indices().t().view(-1, out_height, out_width).contiguous().view(-1, 3)
        output_values = out_sparse._values()

        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 2

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]

        self.out_channels, self.in_channels, self.kernel_h, self.kernel_w = self.weight.shape
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, in_channels, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            for oc in range(self.out_channels):
                conv_value = 0.0
                for ic in range(self.in_channels):
                    for ky in range(self.kernel_size):
                        for kx in range(self.kernel_size):
                            yy = y + ky - self.kernel_size // 2
                            xx = x + kx - self.kernel_size // 2
                            if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                weight = self.weight[oc, ic, ky, kx].item()
                                mask = (padded_indices[0] == b) & (padded_indices[1] == ic) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                                if mask.any():
                                    input_value = values[mask].item()
                                    conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 1
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 0

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // self.in_channels_per_group
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % self.in_channels_per_group
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 6
    out_channels = 12
    kernel_size = 1
    stride = 1
    padding = 1
    groups = 2

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  # assuming square kernels

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Extract sparse indices and values
        indices = x_sparse._indices()
        values = x_sparse._values()
        
        # Apply padding to the sparse tensor
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        # Prepare the output sparse tensor
        output_indices = []
        output_values = []

        # Iterate over the sparse values
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // self.in_channels_per_group
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % self.in_channels_per_group
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres de la couche de convolution
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 1
    groups = 3

    # Création de la couche de convolution
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)

    # Création de l'instance de la transformation conv -> matmul pour tenseur sparse
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    # Exemple de tenseur d'entrée sparse
    input_tensor_dense = torch.randn(1, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    # Passage de l'entrée à travers la transformation matmul pour tenseur sparse
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)

    # Passage de l'entrée à travers la couche de convolution
    conv_output = conv_layer(input_tensor_dense)

    # Vérification de l'égalité des résultats
    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    # Affichage du résultat
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


weight – filters of shape (out_channels,in_channelsgroups,kH,kW)(out_channels,groupsin_channels​,kH,kW)
weight – filters of shape (out_channels,in_channelsgroups,kH,kW)(out_channels,groupsin_channels​,kH,kW)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups
        print(self.groups)

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

     
        indices = x_sparse._indices()
        values = x_sparse._values()
        
      
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

  
        output_indices = []
        output_values = []

        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // (self.in_channels // self.groups)
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % (self.in_channels // self.groups)
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices).t()
        output_values = torch.tensor(output_values)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse


if __name__ == "__main__":
 
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 1
    groups = 3

   
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)
    conv_layer.bias.data=torch.zeros_like(conv_layer.bias.data)
    

  
    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()
    


    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)


    conv_output = conv_layer(input_tensor_dense)


    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())

    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,224,224)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

In [ ]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,224,224)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

zonotope_espilon_sparse_tensor
with torch.no_grad():
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(zonotope_espilon_sparse_tensor)

In [ ]:
matmul_output_sparse

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        indices = x_sparse._indices()
        values = x_sparse._values()
        
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        output_indices = []
        output_values = []

        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // (self.in_channels // self.groups)
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % (self.in_channels // self.groups)
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices, dtype=torch.long).t()
        output_values = torch.tensor(output_values, dtype=torch.float)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

if __name__ == "__main__":
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 2
    padding = 1
    groups = 1

    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)
    conv_layer.bias.data = torch.zeros_like(conv_layer.bias.data)

    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)
    conv_output = conv_layer(input_tensor_dense)

    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        indices = x_sparse._indices()
        values = x_sparse._values()
        
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        output_indices = []
        output_values = []

        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // (self.in_channels // self.groups)
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % (self.in_channels // self.groups)
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                if out_y < 0 or out_y >= out_height or out_x < 0 or out_x >= out_width:
                    continue
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices, dtype=torch.long).t()
        output_values = torch.tensor(output_values, dtype=torch.float)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

if __name__ == "__main__":
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 1
    groups = 1

    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)
    conv_layer.bias.data = torch.zeros_like(conv_layer.bias.data)

    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)
    conv_output = conv_layer(input_tensor_dense)

    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        indices = x_sparse._indices()
        values = x_sparse._values()
        
        padded_indices = indices.clone()
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        output_indices = []
        output_values = []

        # Iterate over spatial indices first
        for i in range(values.size(0)):
            b, c, y, x = padded_indices[:, i]
            if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                continue
            if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                continue

            group = c // (self.in_channels // self.groups)
            oc_start = group * (self.out_channels // self.groups)
            oc_end = oc_start + (self.out_channels // self.groups)
            
            for oc in range(oc_start, oc_end):
                conv_value = 0.0
                ic_in_group = c % (self.in_channels // self.groups)
                for ky in range(self.kernel_size):
                    for kx in range(self.kernel_size):
                        yy = y + ky - self.kernel_size // 2
                        xx = x + kx - self.kernel_size // 2
                        if 0 <= yy < padded_height and 0 <= xx < padded_width:
                            weight = self.weight[oc, ic_in_group, ky, kx].item()
                            mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                            if mask.any():
                                input_value = values[mask].sum().item()
                                conv_value += weight * input_value

                if self.bias is not None:
                    conv_value += self.bias[oc].item()

                out_y = (y - self.kernel_size // 2) // self.stride
                out_x = (x - self.kernel_size // 2) // self.stride
                if out_y < 0 or out_y >= out_height or out_x < 0 or out_x >= out_width:
                    continue
                output_indices.append([b.item(), oc, out_y, out_x])
                output_values.append(conv_value)

        output_indices = torch.tensor(output_indices, dtype=torch.long).t()
        output_values = torch.tensor(output_values, dtype=torch.float)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

if __name__ == "__main__":
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 1
    groups = 3

    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)
    conv_layer.bias.data = torch.zeros_like(conv_layer.bias.data)

    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)
    conv_output = conv_layer(input_tensor_dense)

    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseConv2dToMatMul:
    def __init__(self, conv_layer):
        self.conv_layer = conv_layer
        self.weight = conv_layer.weight
        self.bias = conv_layer.bias
        self.stride = conv_layer.stride[0]
        self.padding = conv_layer.padding[0]
        self.groups = conv_layer.groups

        self.out_channels, self.in_channels_per_group, self.kernel_h, self.kernel_w = self.weight.shape
        self.in_channels = self.in_channels_per_group * self.groups
        self.kernel_size = self.kernel_h  

    def sparse_conv_to_matmul(self, x_sparse):
        batch_size, _, height, width = x_sparse.size()
        out_height = (height + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (width + 2 * self.padding - self.kernel_size) // self.stride + 1

        indices = x_sparse._indices()
        values = x_sparse._values()
        
        padded_indices = indices.clone()cd 
        padded_indices[2:] += self.padding
        padded_height = height + 2 * self.padding
        padded_width = width + 2 * self.padding

        output_indices = []
        output_values = []

        # Iterate over spatial indices first
        for y in range(padded_height):
            for x in range(padded_width):
                if y < self.kernel_size // 2 or y >= padded_height - self.kernel_size // 2:
                    continue
                if x < self.kernel_size // 2 or x >= padded_width - self.kernel_size // 2:
                    continue

                mask_spatial = (padded_indices[2] == y) & (padded_indices[3] == x)
                if not mask_spatial.any():
                    continue

                relevant_indices = padded_indices[:, mask_spatial]
                relevant_values = values[mask_spatial]

                for i in range(relevant_values.size(0)):
                    b, c = relevant_indices[0, i], relevant_indices[1, i]

                    group = c // (self.in_channels // self.groups)
                    oc_start = group * (self.out_channels // self.groups)
                    oc_end = oc_start + (self.out_channels // self.groups)
                    
                    for oc in range(oc_start, oc_end):
                        conv_value = 0.0
                        ic_in_group = c % (self.in_channels // self.groups)
                        for ky in range(self.kernel_size):
                            for kx in range(self.kernel_size):
                                yy = y + ky - self.kernel_size // 2
                                xx = x + kx - self.kernel_size // 2
                                if 0 <= yy < padded_height and 0 <= xx < padded_width:
                                    weight = self.weight[oc, ic_in_group, ky, kx].item()
                                    mask = (padded_indices[0] == b) & (padded_indices[1] == c) & (padded_indices[2] == yy) & (padded_indices[3] == xx)
                                    if mask.any():
                                        input_value = values[mask].sum().item()
                                        conv_value += weight * input_value

                        if self.bias is not None:
                            conv_value += self.bias[oc].item()

                        out_y = (y - self.kernel_size // 2) // self.stride
                        out_x = (x - self.kernel_size // 2) // self.stride
                        if out_y < 0 or out_y >= out_height or out_x < 0 or out_x >= out_width:
                            continue
                        output_indices.append([b.item(), oc, out_y, out_x])
                        output_values.append(conv_value)

        output_indices = torch.tensor(output_indices, dtype=torch.long).t()
        output_values = torch.tensor(output_values, dtype=torch.float)

        output_shape = (batch_size, self.out_channels, out_height, out_width)
        output_sparse = torch.sparse.FloatTensor(output_indices, output_values, output_shape)
        return output_sparse

if __name__ == "__main__":
    in_channels = 3
    out_channels = 12
    kernel_size = 3
    stride = 1
    padding = 1
    groups = 3

    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups)
    conv_layer.bias.data = torch.zeros_like(conv_layer.bias.data)

    sparse_conv_to_matmul = SparseConv2dToMatMul(conv_layer)

    input_tensor_dense = torch.randn(10, in_channels, 5, 5)
    input_tensor_sparse = input_tensor_dense.to_sparse()

    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(input_tensor_sparse)
    conv_output = conv_layer(input_tensor_dense)

    diff = torch.sum(matmul_output_sparse.to_dense() - conv_output)
    print("Différence entre les résultats:", diff.item())
    print("Résultat du produit matriciel pour tenseur sparse:", matmul_output_sparse)


In [ ]:
import sys
sys.path.append('app/src')
sys.path.append('./src')
from zono_sparse_gen import ZonoSparseGeneration
test_input = torch.randn(3,112,112)
_,zonotope_espilon_sparse_tensor = ZonoSparseGeneration(test_input,0.01).total_zono()

print(zonotope_espilon_sparse_tensor)
with torch.no_grad():
    matmul_output_sparse = sparse_conv_to_matmul.sparse_conv_to_matmul(zonotope_espilon_sparse_tensor)
print(matmul_output_sparse)

In [ ]:
!pip install jax

In [ ]:
import torch
import jax
import jax.numpy as jnp
from jax import lax
import numpy as np

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize weights and biases
        self.weights = jax.random.normal(jax.random.PRNGKey(0), (out_channels, in_channels, kernel_size, kernel_size))
        if bias:
            self.bias = jax.random.normal(jax.random.PRNGKey(1), (out_channels,))
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, W, H)
        Returns:
            The result of the convolution as a JAX array
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, W, H = sparse_tensor.shape
        
        # Initialize output shape
        out_height = (W - self.kernel_size + 2 * self.padding) // self.stride + 1
        out_width = (H - self.kernel_size + 2 * self.padding) // self.stride + 1
        output = np.zeros((B, self.weights.shape[0], out_height, out_width))

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, w, h = indices[:, i].tolist()
            value = values[i].item()
            for k in range(self.weights.shape[0]):
                for kh in range(self.kernel_size):
                    for kw in range(self.kernel_size):
                        h_out = (h - kw + self.padding) // self.stride
                        w_out = (w - kh + self.padding) // self.stride
                        if 0 <= h_out < out_height and 0 <= w_out < out_width:
                            output[b, k, h_out, w_out] += value * self.weights[k, c, kh, kw]

        if self.bias is not None:
            output += self.bias[:, None, None]

        return jnp.array(output)

# Example usage:
B, C, W, H = 1, 3, 5, 5
data = torch.sparse_coo_tensor(indices=[[0, 0, 0, 1], [0, 1, 2, 0], [1, 2, 3, 4], [1, 2, 3, 4]],
                               values=[1, 2, 3, 4],
                               size=[B, C, W, H])

conv = SparseConv2D(in_channels=3, out_channels=2, kernel_size=3)
result = conv(data)
print(result)


In [ ]:
import torch
import jax
import jax.numpy as jnp
import numpy as np

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize weights and biases
        self.weights = jax.random.normal(jax.random.PRNGKey(0), (out_channels, in_channels, kernel_size, kernel_size))
        if bias:
            self.bias = jax.random.normal(jax.random.PRNGKey(1), (out_channels,))
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, W, H)
        Returns:
            The result of the convolution as a JAX array
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, W, H = sparse_tensor.shape
        
        # Initialize output shape
        out_height = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        output = np.zeros((B, self.weights.shape[0], out_height, out_width))

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, w, h = indices[:, i].tolist()
            value = values[i].item()
            for k in range(self.weights.shape[0]):
                for kh in range(self.kernel_size):
                    for kw in range(self.kernel_size):
                        h_out = (h - kw + self.padding) // self.stride
                        w_out = (w - kh + self.padding) // self.stride
                        if 0 <= h_out < out_height and 0 <= w_out < out_width:
                            output[b, k, h_out, w_out] += value * self.weights[k, c, kh, kw]

        if self.bias is not None:
            output += self.bias[:, None, None]

        return jnp.array(output)

# Example usage:
B, C, W, H = 100000, 3, 5, 5
data = torch.sparse_coo_tensor(indices=[[0, 0, 0, 0], [0, 1, 2, 0], [1, 2, 3, 4], [1, 2, 3, 4]],
                               values=[1, 2, 3, 4],
                               size=[B, C, W, H])

conv = SparseConv2D(in_channels=3, out_channels=2, kernel_size=3)
result = conv(data)
print(result.shape)


In [ ]:
import torch
import jax
import jax.numpy as jnp

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize weights and biases
        if weights is None:
            self.weights = jax.random.normal(jax.random.PRNGKey(0), (out_channels, in_channels, kernel_size, kernel_size))
        else:
            self.weights = weights
        
        if bias:
            if bias_val is None:
                self.bias = jax.random.normal(jax.random.PRNGKey(1), (out_channels,))
            else:
                self.bias = bias_val
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, W, H)
        Returns:
            The result of the convolution as a JAX array
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, W, H = sparse_tensor.shape
        
        # Initialize output shape
        out_height = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        output = np.zeros((B, self.weights.shape[0], out_height, out_width))

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, w, h = indices[:, i].tolist()
            value = values[i].item()
            for k in range(self.weights.shape[0]):
                for kh in range(self.kernel_size):
                    for kw in range(self.kernel_size):
                        h_out = (h - kw + self.padding) // self.stride
                        w_out = (w - kh + self.padding) // self.stride
                        if 0 <= h_out < out_height and 0 <= w_out < out_width:
                            output[b, k, h_out, w_out] += value * self.weights[k, c, kh, kw]

        if self.bias is not None:
            output += self.bias[:, None, None]

        return jnp.array(output)

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    weights = jnp.array(conv2d.weight.detach().numpy())
    if conv2d.bias is not None:
        bias = jnp.array(conv2d.bias.detach().numpy())
    else:
        bias = None
    
    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, W, H = 100, 3, 5, 5
data = torch.sparse_coo_tensor(indices=[[0, 0, 0, 0], [0, 1, 2, 0], [1, 2, 3, 4], [1, 2, 3, 4]],
                               values=[1., 2., 3., 4.],
                               size=[B, C, W, H])

# Applying the sparse convolution
result = sparse_conv2d(data)
print(result)


In [ ]:
import torch
import jax
import jax.numpy as jnp
import numpy as np

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize weights and biases
        if weights is None:
            self.weights = jax.random.normal(jax.random.PRNGKey(0), (out_channels, in_channels, kernel_size, kernel_size))
        else:
            self.weights = weights
        
        if bias:
            if bias_val is None:
                self.bias = jax.random.normal(jax.random.PRNGKey(1), (out_channels,))
            else:
                self.bias = bias_val
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, W, H)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, W, H = sparse_tensor.shape
        
        # Initialize output shape
        out_height = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        output_values = []
        output_indices = []

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, w, h = indices[:, i].tolist()
            value = values[i].item()
            for k in range(self.weights.shape[0]):
                for kh in range(self.kernel_size):
                    for kw in range(self.kernel_size):
                        h_out = (h - kw + self.padding) // self.stride
                        w_out = (w - kh + self.padding) // self.stride
                        if 0 <= h_out < out_height and 0 <= w_out < out_width:
                            output_values.append(value * self.weights[k, c, kh, kw])
                            output_indices.append([b, k, h_out, w_out])

        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output_values.append(self.bias[k])
                            output_indices.append([b, k, h_out, w_out])

        output_values = np.array(output_values)
        output_indices = np.array(output_indices).T
        
        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output_indices, output_values, size)
        
        return sparse_output

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    weights = jnp.array(conv2d.weight.detach().numpy())
    if conv2d.bias is not None:
        bias = jnp.array(conv2d.bias.detach().numpy())
    else:
        bias = None
    
    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1)
conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, W, H = 150000, 3, 5, 5
data = torch.sparse_coo_tensor(indices=[[0, 0, 0, 0], [0, 1, 2, 0], [1, 2, 3, 4], [1, 2, 3, 4]],
                               values=[1., 2., 3., 4.],
                               size=[B, C, W, H])

# Applying the sparse convolution
result = sparse_conv2d(data)
print(result)


In [ ]:
data_dense = data.to_dense()
print(data_dense.shape)
dense_conv = conv2d(data_dense)
print(dense_conv)

In [ ]:
print(dense_conv-result.to_dense())


In [1]:
import torch
import torch.nn.functional as F
import numpy as np

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
        # Initialize weights and biases
        if weights is None:
            self.weights = torch.randn(out_channels, in_channels, kernel_size, kernel_size)
        else:
            self.weights = torch.tensor(weights, dtype=torch.float32)
        
        if bias:
            if bias_val is None:
                self.bias = torch.randn(out_channels)
            else:
                self.bias = torch.tensor(bias_val, dtype=torch.float32)
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, W, H)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, W, H = sparse_tensor.shape
        out_height = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        output_values = []
        output_indices = []

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, w, h = indices[:, i].tolist()
            value = values[i].item()
            for k in range(self.weights.shape[0]):
                for kh in range(self.kernel_size):
                    for kw in range(self.kernel_size):
                        h_out = (h - kw + self.padding) // self.stride
                        w_out = (w - kh + self.padding) // self.stride
                        if 0 <= h_out < out_height and 0 <= w_out < out_width:
                            output_values.append(value * self.weights[k, c, kh, kw].item())
                            output_indices.append([b, k, h_out, w_out])

        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output_values.append(self.bias[k].item())
                            output_indices.append([b, k, h_out, w_out])

        output_values = torch.tensor(output_values)
        output_indices = torch.tensor(output_indices).T
        
        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output_indices, output_values, size).coalesce()
        
        return sparse_output

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    weights = conv2d.weight.detach().numpy()
    if conv2d.bias is not None:
        bias = conv2d.bias.detach().numpy()
    else:
        bias = None
    
    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1)
conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)


# Creating a sparse tensor for testing
B, C, W, H = 15, 3,224,224
data = torch.sparse_coo_tensor(indices=[[0, 0, 0, 0], [0, 1, 2, 0], [1, 2, 3, 4], [1, 2, 3, 4]],
                               values=[1., 2., 3., 4.],
                               size=[B, C, W, H])

# Applying the sparse convolution
result = sparse_conv2d(data)
print(result)


tensor(indices=tensor([[  0,   0,   0,  ...,   0,   0,   0],
                       [  0,   0,   0,  ..., 127, 127, 127],
                       [  0,   0,   0,  ...,   5,   5,   5],
                       [  0,   1,   2,  ...,   3,   4,   5]]),
       values=tensor([ 0.0047,  0.0740,  0.0823,  ..., -0.1086, -0.7443,
                       0.7202]),
       size=(15, 128, 224, 224), nnz=3072, layout=torch.sparse_coo)


In [40]:
import torch
import torch.nn.functional as F

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups
        self.dilatation = 1
        
        # Initialize weights and biases
        if weights is None:
            self.weights = torch.randn(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            self.weights = torch.tensor(weights, dtype=torch.float32)
        
        if bias:
            if bias_val is None:
                self.bias = torch.randn(out_channels)
            else:
                self.bias = torch.tensor(bias_val, dtype=torch.float32)
        else:
            self.bias = None
        

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, H, W)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()
        
        B, C, H, W = sparse_tensor.shape
        out_height = (H + 2 * self.padding - self.dilatation*(self.kernel_size-1)-1) // self.stride + 1
        out_width = (W + 2 * self.padding - self.dilatation *(self.kernel_size-1)-1) // self.stride + 1
        output_values = []
        output_indices = []
        print(self.weights.shape)
        print('out height', out_height)
        print('out_width',out_width)

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, h, w = indices[:, i].tolist()
            value = values[i].item()
        
            for kh in range(self.kernel_size):
                for kw in range(self.kernel_size):
                    h_out = (h - kh + 2*self.padding) // self.stride
                    w_out = (w - kw + 2*self.padding) // self.stride
                    if 0 <= h_out < out_height and 0 <= w_out < out_width:
                        for k in range(self.weights.shape[0]):
                            output_values.append(value * self.weights[k, (c % (C // self.groups)), kh, kw].item())
                            output_indices.append([b, k, h_out, w_out])

        # Apply bias
        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output_values.append(self.bias[k].item())
                            output_indices.append([b, k, h_out, w_out])

        output_values = torch.tensor(output_values)
        output_indices = torch.tensor(output_indices).T
        
        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output_indices, output_values, size)
        
        return sparse_output

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance
    
    Args:
        conv2d: instance of torch.nn.Conv2d
    
    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    groups = conv2d.groups
    weights = conv2d.weight.detach().numpy()
    if conv2d.bias is not None:
        bias = conv2d.bias.detach().numpy()
    else:
        bias = None
    
    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=2, padding=2, groups=1)

conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, H, W = 1, 3, 4, 4
indices = torch.tensor([[0, 0, 0, 1], [0, 1, 2, 0], [2, 1, 3, 4], [2, 2, 3, 4]])
values = torch.tensor([1., 2., 3., 4.])
data = torch.sparse_coo_tensor(indices, values, size=[B, C, H, W])

# Applying the sparse convolution
result = sparse_conv2d(data)

# For comparison, dense convolution on a dense version of the sparse tensor
dense_data = data.to_dense()
conv2d_result = conv2d(dense_data)

print("SparseConv2D result (sparse):", result.to_dense())
print("Conv2D result (dense):", conv2d_result)


torch.Size([3, 3, 3, 3])
out height 3
out_width 3
SparseConv2D result (sparse): tensor([[[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.2160],
          [ 0.0000,  0.0000,  0.7975]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0127],
          [ 0.0000,  0.0000, -1.0210]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.3500],
          [ 0.0000,  0.0000,  0.1905]]]])
Conv2D result (dense): tensor([[[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0286,  0.3873],
          [ 0.0000,  0.0736,  0.1582]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.3039, -0.2102],
          [ 0.0000,  0.0895, -0.2971]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.3208, -0.3539],
          [ 0.0000,  0.1880, -0.1693]]]], grad_fn=<ConvolutionBackward0>)


In [ ]:
import torch
import torch.nn.functional as F

class SparseConv2D:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1, bias=True, weights=None, bias_val=None):
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.groups = groups

        # Initialize weights and biases
        if weights is None:
            self.weights = torch.randn(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            self.weights = torch.tensor(weights, dtype=torch.float32)

        if bias:
            if bias_val is None:
                self.bias = torch.randn(out_channels)
            else:
                self.bias = torch.tensor(bias_val, dtype=torch.float32)
        else:
            self.bias = None

    def __call__(self, sparse_tensor):
        """
        Args:
            sparse_tensor: a torch sparse tensor in COO format with shape (B, C, H, W)
        Returns:
            The result of the convolution as a sparse tensor in COO format
        """
        coo = sparse_tensor.coalesce()
        values = coo.values()
        indices = coo.indices()

        B, C, H, W = sparse_tensor.shape
        out_height = (H + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_width = (W + 2 * self.padding - self.kernel_size) // self.stride + 1
        output_values = []
        output_indices = []

        # Iterate over the non-zero elements in the sparse tensor
        for i in range(values.shape[0]):
            b, c, h, w = indices[:, i].tolist()
            value = values[i].item()
            for kh in range(self.kernel_size):
                for kw in range(self.kernel_size):
                    h_out = (h + self.padding - kh) // self.stride
                    w_out = (w + self.padding - kw) // self.stride
                    if 0 <= h_out < out_height and 0 <= w_out < out_width:
                        for k in range(self.weights.shape[0]):
                            output_values.append(value * self.weights[k, c % (C // self.groups), kh, kw].item())
                            output_indices.append([b, k, h_out, w_out])

        # Apply bias
        if self.bias is not None:
            for b in range(B):
                for k in range(self.weights.shape[0]):
                    for h_out in range(out_height):
                        for w_out in range(out_width):
                            output_values.append(self.bias[k].item())
                            output_indices.append([b, k, h_out, w_out])

        output_values = torch.tensor(output_values)
        output_indices = torch.tensor(output_indices).T

        size = (B, self.weights.shape[0], out_height, out_width)
        sparse_output = torch.sparse_coo_tensor(output_indices, output_values, size).coalesce()

        return sparse_output

def conv2d_to_sparseconv2d(conv2d):
    """
    Transforms a torch.nn.Conv2d instance to a SparseConv2D instance

    Args:
        conv2d: instance of torch.nn.Conv2d

    Returns:
        instance of SparseConv2D
    """
    in_channels = conv2d.in_channels
    out_channels = conv2d.out_channels
    kernel_size = conv2d.kernel_size[0]
    stride = conv2d.stride[0]
    padding = conv2d.padding[0]
    groups = conv2d.groups
    weights = conv2d.weight.detach().numpy()
    if conv2d.bias is not None:
        bias = conv2d.bias.detach().numpy()
    else:
        bias = None

    return SparseConv2D(in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=(bias is not None), weights=weights, bias_val=bias)

# Example usage:
conv2d = torch.nn.Conv2d(in_channels=3, out_channels=18, kernel_size=3, stride=1, padding=1, groups=3)
conv2d.bias = None
sparse_conv2d = conv2d_to_sparseconv2d(conv2d)

# Creating a sparse tensor for testing
B, C, H, W = 1, 3, 5, 5
indices = torch.tensor([[0, 0, 0, 1], [0, 1, 2, 0], [2, 1, 3, 4], [2, 2, 3, 4]])
values = torch.tensor([1., 2., 3., 4.])
data = torch.sparse_coo_tensor(indices, values, size=[B, C, H, W])

# Applying the sparse convolution
result = sparse_conv2d(data)

# For comparison, dense convolution on a dense version of the sparse tensor
dense_data = data.to_dense()
conv2d_result = conv2d(dense_data)

print("SparseConv2D result (sparse):", result.to_dense())
print("Conv2D result (dense):", conv2d_result)


In [3]:
import torch

def sparse_tensor_stats(sparse_tensor):
    # Vérifie si le tenseur est au format COO
    if sparse_tensor.layout != torch.sparse_coo:
        raise ValueError("Le tenseur doit être au format COO")

    # Calculer le nombre total d'éléments dans le tenseur
    total_elements = sparse_tensor.size(0) * sparse_tensor.size(1)
    
    # Calculer le nombre d'éléments non nuls
    non_zero_elements = sparse_tensor._nnz()
    
    # Calculer le pourcentage de valeurs non nulles
    non_zero_percentage = (non_zero_elements / total_elements) * 100
    
    # Calculer la mémoire utilisée par le tenseur dense
    dense_memory = total_elements * sparse_tensor.dtype.itemsize
    
    # Calculer la mémoire utilisée par le tenseur sparse
    values_memory = sparse_tensor.values().numel() * sparse_tensor.values().element_size()
    indices_memory = sparse_tensor.indices().numel() * sparse_tensor.indices().element_size()
    sparse_memory = values_memory + indices_memory
    
    # Calculer le gain en pourcentage de mémoire
    memory_gain_percentage = ((dense_memory - sparse_memory) / dense_memory) * 100
    
    return non_zero_percentage, memory_gain_percentage

# Exemple d'utilisation
indices = torch.tensor([[0, 1, 2, 3], [0, 1, 2, 3]])
values = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
sparse_tensor = torch.sparse_coo_tensor(indices, values, (50, 50)).coalesce()

non_zero_percentage, memory_gain_percentage = sparse_tensor_stats(sparse_tensor)
print(f"Pourcentage de valeurs non nulles: {non_zero_percentage:.2f}%")
print(f"Gain en pourcentage de mémoire: {memory_gain_percentage:.2f}%")


Pourcentage de valeurs non nulles: 0.16%
Gain en pourcentage de mémoire: 99.20%


In [4]:
x= torch.zeros(3,2,24,24)
torch.sum(x,dim=0)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])